#install hazm

In [ ]:
pip install hazm

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 892.6/892.6 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.1/292.1 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 11.8 MB/s eta 0:00:00
  Created wheel for flashtext: filename=flashtext-2.7-py2.py3-none-any.whl size=9300 sha256=cde837c3896c50bf05cda1c71492f69e1a7abc175eac0dcfb844daed058a57ef
  Stored in directory: /root/.cache/pip/wheels/49/20/47/f03dfa8a7239c54cbc44ff7389eefbf888d2c1873edaaec888
Successfully built flashtext
  Attempting uninstall: numpy
   

In [ ]:
!pip install parsivar

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 15.9 MB/s eta 0:00:00


In [ ]:
!pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 MB 11.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.4 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.15-cp311-cp311-linux_x86_64.whl size=4397851 sha256=5b80aaa38b784c952f8be7ada8b1eb2d9b014e20cf0e929ff2a2675343be06b1
  Stored in directory: /root/.cache/pip/wheels/34/62/22/63039c8c4c8da7ff68e81b8357e64c199faff9df9a2b5e5e3b
Successfully built llama-cpp-python


#imports

In [ ]:
from hazm import Normalizer, word_tokenize
import pandas as pd
import re

In [ ]:
from parsivar import Normalizer, SpellCheck

In [ ]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import login
from huggingface_hub import hf_hub_download
from llama_cpp import Llama
import time
import csv

#Texts

In [ ]:
# Raw Farsi sentences (you can add your 20 samples here)
texts = [
    "مي خواهم كتاب را بخوانم",
    "سلام دكتر. فشار خونم خيلي ًبالاست !",
    "دارو را   روزي سه بر بخور",
    "از ديروز سر درد دارم 不客气 و هيچ دارويي نخوردم",
    "چك كردم ولي قند خون نرمال بود",
    "سلام دکتر 😊 دارو رو روزی 3 بار @ صبح‌ها بخور، اوکی؟ ❤️ #ممنون",
]

In [ ]:
the_texts = [
    "برای درمان تب و بدن‌درد،  باید هر 6 ساعت یک عد قرص استامینووفن 500 مصرف کند و هر 12 ساعت یک کپسول آموکسی‌یلین 500 بعد از غذا مصرف شود.خانم کرمی",
    "آقای صادقی به دلیل باید روزی یک عدد قرص آسیترومایسین 250 به مدت سه روز همراه صبحانه مصرف کند سینوسیت حاد و قطره ونیل‌افرین را هر 8 ساعت در هر سوراخ بینی بچکاند.",
    "خانم عسگری به دلیل آلرسی فصلی باید روزی دو عدد قرص سیتریسین همراه غذا مصرف کند و شب‌ها  شربت دیفن‌هیدراین قبل از خواب یک قاشق میل نماید.",
    "کودک هفت‌ساله‌ای که دچار عفونت گلو شده است باید هر 8 ساعت 5 شربت فالکسین مصرف کند سی‌سی و هر 6 ساعت یک قاشق شربت استامینوفن برای تب تجویزه شده است.",
    "آقای حسام برای زفلاکس معده باید شب‌هاقرص واموتیدین 40 قبل از خواب  یک عدد و همراه ناهار یک قرص رانیتیدین مصرف کند.",
    "جهت درمان سرماخوردگی، خانم آقاجانی باید روزی یه بار لوراتادین قرص پس از غذا مصرف کند و شب‌ها قبل از خواب یک قاشق شربت دیفندرامین نیز میل نماید.",
    "خانم فرمند با تشخیص برونخیت باید روزی یک عدد قرص آزیترومایسین با معده حالی و هر 80 ساعت یک عدد استامینوفن 500 مصرف نماید قرص .",
    "روزی دو باربرای درد مواصل، آقای تقوی باید قرص ایبوپروفن 40 را هر 8 ساعت بعد از غذا و نابروکسن 250 را  همراه یک لیوان آب میل کند.",
    "کودک ده‌ساله‌ای که کم‌خونی دارد باید روزی یک قرص آن همراه صبحانه و شربت آسکوربیک اسید نیز روزی دو بار برای جذب بهتر .",
    "خانم مرادی که دچار حساسیت پوستی است باید روزی یک عدد قرص لوراتادین صبح‌ها و شب‌ها پماد هیدروکورنیزون را روی نواحی آسیب‌ بالد.",
    "برای درمان عفونت مجازی ادراری، آقای خلیلی باید هر 112 ساعت یک کپسول سفالسین 500 و روزی سه بار قرص مترونیداول همراه غذا .",
    "خانم داوری که با اسال شدید و التهاب  باید روزی دو بار قرص لوپرامید و هر روز صبح یک عدد قرص دگزامتازون . مصرف مایات فراموش شود.",
    "کودک سه‌ساله با گلودرد ویروسی باید هر 6 ساعت یک قاشق شربت استامینوفن و هر 12 ساعت یک قاشق شربت هیستامین بعد از غذا .",
    "آقای رفیعی با آلرژی تنفسی باید روزی یک عدد قرص فکسوونادین با شکم خالی و شب‌ها قبل از خواب شربت دیفن‌هیدرامین .",
    "خانم اجباری که دچار کمبود ویتامین C شده است باید شربت آسکوربیک سید را روزی دو بار بعد از غذا و قرص ژینک را همراه وعده شام مصرف کند.",
    "برای بهبود خواب، آقای شایان باید هر شب قبل از خواب یک عدد قرص ظولپیدم 10 مصرف کند و درصورت اضطراب، صبح‌ها یک قرص الپرازولان با معده خالی میل نماید.",
    "کودک هشت‌ساله با اورتیت میانی باید هر 8 ساعت 5 سی‌سی شربت سفیکسلیم و شب‌ها قبل از خواب یک قاشق شربت دیفن‌هیدرامین  شود.",
    "خانم نظری برای رفلاکس باید صبح‌ها ناشتا یک عدد قرص امپرازول و شب‌ها بعد از شام یک قرص فاموتیدین .",
    "خانم نادری با سردرد مرمن باید روزی هر 66 ساعت یک عدد قرص استامینوفن 5000 مصرف کند  در صورت شدت درد، پماد موضعی منتول روی شقیشقه‌ها.",
    " برای درمان عفونت شدید، آقای پرویزی باید روزی یک عدد قرص آسیترومایسین و هر 12 ساعت یک عدد آمپول چنتامایسین تزریقی طبق دستور زشک بزند."
  ]

#cleaning

In [ ]:
def clean_symbols_and_emojis(text):

    # Define allowed punctuation: Persian and some general
    allowed_punct = "،.؟؛:!"

    # Build regex pattern:
    # Pattern to match any character NOT in:
    # - Persian Unicode range \u0600-\u06FF
    # - English letters a-zA-Z
    # - Digits 0-9 and Persian digits \u06F0-\u06F9
    # - Half-space \u200c
    # - Whitespace \s
    # - Allowed punctuation
    pattern = fr"[^a-zA-Z0-9\u06F0-\u06F9\u0600-\u06FF\u200c\s{re.escape(allowed_punct)}]"

    # Remove disallowed characters (emojis, symbols, etc.)
    cleaned = re.sub(pattern, "", text)

    return cleaned


#Normalizing

In [ ]:
# Initialize hazm normalizer
normalizer = Normalizer()

def normalize_farsi_sentences(sentences):
    """
    Normalize a list of Farsi sentences using hazm.
    """
    normalized = []
    for sentence in sentences:
        cleaned = normalizer.normalize(sentence)
        normalized.append(cleaned)
    return normalized


#applying cleaning and normalizing

In [ ]:
# Clean + Normalize pipeline
cleaned_texts = [clean_symbols_and_emojis(t) for t in the_texts]
normalized_texts = [normalizer.normalize(t) for t in cleaned_texts]

# Show original, cleaned, and normalized
for i, (orig, clean, norm) in enumerate(zip(the_texts, cleaned_texts, normalized_texts), 1):
    print(f"{i}. Original   : {orig}")
    print(f"   Cleaned    : {clean}")
    print(f"   Normalized : {norm}")
    print("-" * 50)

1. Original   : برای درمان تب و بدن‌درد،  باید هر 6 ساعت یک عد قرص استامینووفن 500 مصرف کند و هر 12 ساعت یک کپسول آموکسی‌یلین 500 بعد از غذا مصرف شود.خانم کرمی
   Cleaned    : برای درمان تب و بدن‌درد،  باید هر 6 ساعت یک عد قرص استامینووفن 500 مصرف کند و هر 12 ساعت یک کپسول آموکسی‌یلین 500 بعد از غذا مصرف شود.خانم کرمی
   Normalized : برای درمان تب و بدن‌درد ، باید هر 6 ساعت یک عد قرص استامینووفن 500 مصرف کند و هر 12 ساعت یک کپسول آموکسی‌یلین 500 بعد از غذا مصرف شود . خانم کرمی
--------------------------------------------------
2. Original   : آقای صادقی به دلیل باید روزی یک عدد قرص آسیترومایسین 250 به مدت سه روز همراه صبحانه مصرف کند سینوسیت حاد و قطره ونیل‌افرین را هر 8 ساعت در هر سوراخ بینی بچکاند.
   Cleaned    : آقای صادقی به دلیل باید روزی یک عدد قرص آسیترومایسین 250 به مدت سه روز همراه صبحانه مصرف کند سینوسیت حاد و قطره ونیل‌افرین را هر 8 ساعت در هر سوراخ بینی بچکاند.
   Normalized : آقای صادقی به دلیل باید روزی یک عدد قرص آسیترومایسین 250 به مدت سه روز همراه صبحانه مصرف کند سینو

#creat a csv

In [ ]:
# Create DataFrame
df = pd.DataFrame({
    "Original": the_texts,
    "Cleaned": cleaned_texts,
    "Normalized": normalized_texts
})
df.to_csv("farsi_cleaned_normalized.csv", index=False, encoding="utf-8-sig")
print("✅ Exported to farsi_cleaned_normalized.csv")

✅ Exported to farsi_cleaned_normalized.csv


# loading and testing model

In [7]:
!pip install aiofiles faspell

In [11]:
# --- تنظیمات سریع ---
LEXICON_PATH = "fa_words.txt"  # اگر وجود نداشته باشد، به‌طور خودکار از hazm دانلود می‌شود
HAZM_WORDS_URL = "https://raw.githubusercontent.com/sobhe/hazm/master/hazm/data/words.dat"

IN_CSV  = "farsi_cleaned_normalized.csv"
IN_COL  = "Normalized"
OUT_CSV = "farsi_async_faspell_corrected_on_normalized.csv"
OUT_COL = "async_faspell_on_Normalized"

# --- اگر در Colab هستید، اول این را اجرا کنید:
# !pip install -q aiofiles faspell

import os
import re
import asyncio
import pandas as pd
from typing import List

# تلاش برای وارد کردن پکیج‌ها؛ پیام راهنما اگر نصب نشده باشد
try:
    from faspell.spell_checker import SpellChecker  # async_faspell
except Exception as e:
    raise RuntimeError(
        "پکیج faspell نصب نیست. لطفاً اجرا کنید: pip install faspell aiofiles"
    ) from e

import aiofiles

# --- ابزار دانلود امن (بدون وابستگی به !wget) ---
def ensure_lexicon(path: str, fallback_url: str) -> str:
    """
    اگر فایل لغت‌نامه موجود نبود، از fallback_url دانلود می‌کند و در path ذخیره می‌کند.
    در پایان مسیر فایل موجود را برمی‌گرداند.
    """
    if os.path.isfile(path) and os.path.getsize(path) > 0:
        return path
    # دانلود hazm words.dat و ذخیره به عنوان fa_words.txt
    try:
        import urllib.request
        print(f"⬇️  در حال دانلود لغت‌نامه از {fallback_url} ...")
        with urllib.request.urlopen(fallback_url) as resp:
            data = resp.read().decode("utf-8")
        # hazm/words.dat یک واژه در هر خط است؛ همان را ذخیره می‌کنیم
        with open(path, "w", encoding="utf-8") as f:
            f.write(data)
        if os.path.getsize(path) == 0:
            raise IOError("دانلود انجام شد ولی فایل خالی است.")
        print(f"✅ لغت‌نامه در «{path}» ذخیره شد.")
        return path
    except Exception as e:
        raise FileNotFoundError(
            f"نتوانستم لغت‌نامه را پیدا/دانلود کنم. یا «{path}» را دستی بسازید، "
            f"یا اتصال اینترنت را بررسی کنید. خطا: {e}"
        )

# --- الگوهای کلمه فارسی و توکن‌سازی ساده ---
PERSIAN_ARABIC_CHARS = r"\u0600-\u06FF\u0750-\u077F\u08A0-\u08FF"
WORD_RE = re.compile(rf"[{PERSIAN_ARABIC_CHARS}]+(?:[{PERSIAN_ARABIC_CHARS}‌\-]+)*")

def is_persian_word(w: str) -> bool:
    w = w.strip(".,!?؛،:()[]{}«»\"'…")
    return bool(w) and bool(re.fullmatch(rf"[{PERSIAN_ARABIC_CHARS}‌\-]+", w))

def tokenize_sentence_keep_punct(s: str) -> List[str]:
    # تقسیم خیلی ساده بر اساس فاصله؛ علائم حفظ می‌شوند
    return s.split()

def rebuild_sentence(tokens: List[str]) -> str:
    txt = " ".join(tokens)
    # اصلاح فاصله‌ها پیرامون علائم رایج
    txt = re.sub(r"\s+([،؛,.!?؟…])", r"\1", txt)
    txt = re.sub(r"«\s+", "«", txt)
    txt = re.sub(r"\s+»", "»", txt)
    return txt

import asyncio

async def load_spellchecker(lexicon_path: str) -> "SpellChecker":
    # مثل قبل، ولی ok
    import aiofiles
    from faspell.spell_checker import SpellChecker
    async with aiofiles.open(lexicon_path, "r", encoding="utf-8") as f:
        lex_text = await f.read()
    return SpellChecker(lex_text)

async def correct_token(sp, token: str) -> str:
    core = token.strip(".,!?؛،:()[]{}«»\"'…")
    if not core or not re.fullmatch(rf"[{PERSIAN_ARABIC_CHARS}‌\-]+", core):
        return token

    # سازگار با هر دو پیاده‌سازی: اگر خروجی coroutine بود await کن، وگرنه مستقیم استفاده کن
    res = sp.correct(core)
    if asyncio.iscoroutine(res):
        suggs = await res
    else:
        suggs = res  # sync list

    if not suggs:
        return token
    best = suggs[0]
    if best == core:
        return token

    prefix = token[:len(token) - len(token.lstrip("«({[\"'"))]
    suffix = token[len(token.rstrip(".,!?؛،:)}]»\"'…")):]
    return f"{prefix}{best}{suffix}"

async def correct_sentence(sp: SpellChecker, sentence: str) -> str:
    tokens = tokenize_sentence_keep_punct(sentence)
    tasks = [correct_token(sp, tk) for tk in tokens]
    fixed_tokens = await asyncio.gather(*tasks)
    return rebuild_sentence(fixed_tokens)

async def process_csv(
    lexicon_path: str,
    in_csv: str,
    in_col: str,
    out_csv: str,
    out_col: str,
    batch_size: int = 256
):
    # 1) بارگذاری لغت‌نامه
    sp = await load_spellchecker(lexicon_path)

    # 2) خواندن CSV
    df = pd.read_csv(in_csv)
    if in_col not in df.columns:
        raise KeyError(f"ستون «{in_col}» در CSV پیدا نشد. ستون‌ها: {list(df.columns)}")
    texts = df[in_col].fillna("").astype(str).tolist()

    # 3) اجرای اصلاحات به‌صورت دسته‌ای
    corrected = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        tasks = [correct_sentence(sp, t) for t in batch]
        corrected.extend(await asyncio.gather(*tasks))

    # 4) ذخیره خروجی
    df[out_col] = corrected
    df.to_csv(out_csv, index=False, encoding="utf-8-sig")
    print(f"✅ Saved to {out_csv}")

# ---------- اجرای یک‌کلیکی ----------
# 1) اطمینان از وجود لغت‌نامه (در صورت نبود، دانلود از hazm)
lex_path = ensure_lexicon(LEXICON_PATH, HAZM_WORDS_URL)

# 2) اجرای پردازش
# در نوت‌بوک‌هایی که event loop فعال دارند:
try:
    import nest_asyncio, sys
    nest_asyncio.apply()
    loop = asyncio.get_event_loop()
    if loop.is_running():
        # اگر لوپ در حال اجراست (Jupyter)، از create_task استفاده می‌کنیم
        task = loop.create_task(process_csv(lex_path, IN_CSV, IN_COL, OUT_CSV, OUT_COL))
        await task  # در Colab/Notebook، await در سلول مجازه
    else:
        loop.run_until_complete(process_csv(lex_path, IN_CSV, IN_COL, OUT_CSV, OUT_COL))
except RuntimeError:
    # محیط‌های معمولی (اسکریپت)
    asyncio.run(process_csv(lex_path, IN_CSV, IN_COL, OUT_CSV, OUT_COL))


✅ Saved to farsi_async_faspell_corrected_on_normalized.csv
